### Imports

In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

import config
from src import data_ingestion
from src import data_providers
from src import benchmark
from src import portfolio

### Load Data

In [2]:
# --- Load Data ---
master_log = data_ingestion.create_master_log([
    config.INPUT_DATA_DIR / "us_mkt_transactions.csv",
    config.INPUT_DATA_DIR / "exus_mkt_transactions.csv"
]).set_index('Date')

master_log

Successfully loaded log: us_mkt_transactions.csv
Successfully loaded log: exus_mkt_transactions.csv


,Type,Symbol,Quantity,Price,Amount,Commission,Currency,Description,Market,Source
Date,,,,,,,,,,
2023-02-16,Net Deposit,NaN,NaN,NaN,271.57,NaN,USD,ID: 37fdafdc-d707-42fa-ba74-98b30cf9ab2a - DT2...,US Market,Sarwa Trade
2023-02-17,buy,PERI,0.407270,33.340000,-13.58,NaN,USD,Trade Entry,US Market,Sarwa Trade
2023-02-17,buy,TSLA,0.202666,201.020000,-40.74,NaN,USD,Trade Entry,US Market,Sarwa Trade
2023-02-17,buy,VOO,0.218967,372.070000,-81.47,NaN,USD,Trade Entry,US Market,Sarwa Trade
2023-03-29,Net Dividend,VOO,NaN,NaN,0.24,NaN,USD,"Cash DIV @ 1.4874, Pos QTY: 0.218966527, Rec D...",US Market,Sarwa Trade
...,...,...,...,...,...,...,...,...,...,...
2025-07-24,buy,BRK-B,0.124000,481.935484,-59.76,1.0,USD,Purchase of BRK-B,US Market,Sarwa Trade - Interim
2025-07-24,buy,BTC-USD,0.000162,120615.384615,-19.60,NaN,USD,NaN,US Market,Sarwa Crypto
2025-07-24,Net Deposit,NaN,NaN,NaN,271.57,NaN,USD,Bank Deposit,US Market,Sarwa Trade - Interim


### Project Variables

In [3]:
# --- Initialize Project Variables ---
start_date, end_date, date_range, last_market_day = config.project_dates(master_log.index)

# User-defined constants are accessed directly
benchmark_index = config.BENCHMARK_INDEX
tax_rate = config.TAX_RATE

In [4]:
start_date
end_date
last_market_day

Timestamp('2025-07-29 00:00:00')

In [14]:
data_providers.yf_hist("STIP", start_date, last_market_day)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2023-02-16,90.389077,90.500339,90.333450,90.351990,864100,0.0,0.0,0.0
2023-02-17,90.231473,90.398363,90.222200,90.342735,855300,0.0,0.0,0.0
2023-02-21,90.268547,90.351988,90.185099,90.268547,804000,0.0,0.0,0.0
2023-02-22,90.398333,90.453960,90.138728,90.166542,976000,0.0,0.0,0.0
2023-02-23,90.166559,90.333448,90.166559,90.268547,959800,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2025-07-23,102.959999,102.970001,102.830002,102.849998,539800,0.0,0.0,0.0
2025-07-24,102.790001,102.860001,102.779999,102.839996,423300,0.0,0.0,0.0
2025-07-25,102.870003,102.879997,102.820000,102.830002,451000,0.0,0.0,0.0


### `portfolio.py` Test

In [ ]:
# In your main script or notebook (e.g., src/portfolio.py)

# Get a unique list of every stock and its designated market from the log
symbol_market_pairs = master_log[['Symbol', 'Market']].dropna().drop_duplicates()

# Initialize empty DataFrames for price and split data
raw_splits = pd.DataFrame(index=date_range, columns=symbols)
holdings['price'] = pd.DataFrame(index=date_range, columns=symbols)

# Loop through the unique pairs to fetch data using the correct provider
for _, row in symbol_market_pairs.iterrows():
    symbol = row['Symbol']
    market = row['Market']

    print(f"Fetching data for {symbol} on {market}...")
    hist = None

    if market == 'US Market':
        # Use yfinance for all US market transactions
        hist = data_providers.yfinance_hist(symbol, start_date, end_date, last_market_day)
    else:
        # Use Twelve Data for all other markets
        # The 'Market' value (e.g., "ADX", "DFM") is used as the exchange code
        hist = data_providers.get_twelve_data_history(config.API_KEY_TWELVE_DATA, symbol, exchange=market)

    # Populate the main dataframes with the fetched data
    if hist is not None and not hist.empty:
        # Use .reindex() to align the data with your full date_range
        holdings['price'][symbol] = hist['Close'].reindex(date_range).ffill()
        raw_splits[symbol] = hist['Stock Splits'].reindex(date_range)

# Fill any remaining NaNs after fetching data for all symbols
raw_splits = raw_splits.fillna(0.0)

# Now, you can proceed with your existing holdings calculation loop
# ...